### 1. Collect / Preprocess Data

In [1]:
from datasets import load_dataset, DatasetDict

In [2]:
ds_train = load_dataset("lex_glue","case_hold", split = "train")
ds_valid = load_dataset("lex_glue","case_hold", split = "validation")

Found cached dataset lex_glue (/home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a)
Found cached dataset lex_glue (/home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a)


In [8]:
raw_datasets = DatasetDict(
   {"train": ds_train.shuffle().select(range(5000)),
    "valid": ds_valid.shuffle().select(range(500))
   }
)

In [17]:
for key in raw_datasets['train'][0]:
    print(key)

context
endings
label


In [22]:
import torch
torch.cuda.is_available()

True

- GPT-2, GPT-3에서는 컨텍스트 크기가 각각 1024, 2048. LLaMA도 찾아보니 2048이라 함. 출처)https://news.ycombinator.com/item?id=35186185. 일단은 128 토큰으로 정해보자.
- 여기서 중요한 것은, return_overflowing_tokens 옵션을 사용해서 전체 입력을 토큰화 하고 여러 청크로 분할하는 작업이다. 또 return_length 옵션을 사용해 생성된 각 청크의 길이를 자동으로 반환할 것임. 마지막 청크는 드랍시켜줄 것.
- context_length를 128 > 2048로 바꿔보면 청크의 개수가 늘어나고, 각 청크의 길이는 줄어들게 됨

In [9]:
from transformers import AutoTokenizer

In [10]:
context_length = 128
tokenizer = AutoTokenizer.from_pretrained('./llama', add_eos_token = True)

In [11]:
outputs = tokenizer(
    raw_datasets['train'][:5]['context'],
    truncation = True,
    max_length = context_length,
    return_overflowing_tokens = True,
    return_length = True)

In [125]:
len(outputs['input_ids'][0])

128

In [126]:
print(f'Input IDs length: {len(outputs["input_ids"])}')
print(f'Input chunk lengths: {(outputs["length"])}')
print(f'Chunk mapping: {outputs["overflow_to_sample_mapping"]}') # 어떤 청크가 어떤 입력 샘플에 속하는 지 재구성할 수도 있음

Input IDs length: 11
Input chunk lengths: [128, 100, 128, 97, 128, 128, 128, 128, 40, 128, 94]
Chunk mapping: [0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4]


In [83]:
outputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'])

In [ ]:
output

- 더 효율적인 data preparation을 위해서는 eos_token_id 토큰을 사용해서 단일 배치 내의 모든 토큰화된 샘플을 결합한 다음 결합된 시퀀스에서 청킹을 수행하는 것. 

In [12]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(
    './llama',
    vocab_size = len(tokenizer),
    n_ctx = context_length,
    bos_token_id = tokenizer.bos_token_id,
    eos_token_id = tokenizer.eos_token_id)

In [128]:
config

LlamaConfig {
  "_name_or_path": "./llama",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.29.1",
  "use_cache": true,
  "vocab_size": 32000
}

In [13]:
def tokenize(element):
    outputs = tokenizer(
    element['context'],
    truncation = True,
    max_length = context_length,
    return_overflowing_tokens = True,
    return_length = True
    )
    input_batch = []
#     input_batch.append(outputs['input_ids'])
#     return {"input_ids": input_batch} 
    for length, input_ids in zip(outputs['length'], outputs['input_ids']):
        if length == context_length:
            input_batch.append(input_ids) # dropping
    return {"input_ids": input_batch} 
    
tokenized_datasets = raw_datasets.map(
    tokenize, batched = True, remove_columns = raw_datasets['train'].column_names
) # batched = True: True로 설정되, 호출되는 순간마다 여러 개의 예제로 구성된 하나의 배치(batch)가 한번에 map 함수에 입력됨.배치 크기는 별도로 설정이 가능하며, 디폴트 값은 1000. 

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [186]:
#[tokenized_datasets['train'][i] for i in range(5)]
len(tokenized_datasets['train'])

8442

In [189]:
len(tokenized_datasets['train'][0]['input_ids'])

128

In [14]:
from transformers import LlamaForCausalLM
#model = LlamaForCausalLM(config, load_in_8bit = True) # from_pretrained()함수를 사용하지 않고 모델을 직접 초기화 함
model = LlamaForCausalLM.from_pretrained('./llama', load_in_8bit = True, device_map = 'auto')

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/laal_intern003/anaconda/envs/legal-master did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/run/user/10100/vscode-ipc-3405e988-41a9-43ff-ab2f-298fa0e77142.sock')}
  warn(msg)
/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_se

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [109]:
model_size = sum(t.numel() for t in model.parameters())

In [111]:
print(f'LLaMA size: {model_size/1000**2:.1f}M parameters')

LLaMA size: 6738.4M parameters


- 우리모델에는 optimize해야 할 67억 3840만개의 parameter가 있다. 학습을 시작하기 전에 배치 생성을 처리할 datacollator를 설정해야 한다.
- datacollator가 하는 일: 1) 배치 구성 2) 패딩 3) LM label 생성. 참고로 datacollator는 학습 중에 입력을 자동으로 생성하기 때문에 input_ids를 복제할 필요가 없다.
- DataCollatorForLanguageModeling을 사용. 이 모듈은 MLM, CLM을(mlm=False) 모두 지원.

In [15]:
from transformers import DataCollatorForLanguageModeling

print(f'{tokenizer.bos_token}, {tokenizer.eos_token}')
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False) # causal language model

<s>, </s>


In [118]:
data_collator

DataCollatorForLanguageModeling(tokenizer=LlamaTokenizerFast(name_or_path='./llama', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False), mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [176]:
data_collator([tokenized_datasets['train'][i] for i in range(5)])
# input_ids만 넣어줬는데도 attention mask labels이 붙은 채로 텐서의 형태로 통일되어 리턴시켜줌

{'input_ids': tensor([[    1,   505,  1303,   278, 22569,  9245, 30010, 29879,   766,  4838,
           291,   653,   740,  4251,   408,   972,  5414, 13047,   304,  8820,
           393,   526,  1185,   329,  2015,  1891,  1363,   896,   526,   443,
          3075,  5008,   284, 29892,   410,  7588,  2580,   491,  1002,  1082,
         29892,   470, 13461,   278,  6874,   310,   385,  6221, 30010, 29879,
         14329,  3178,   498,  1280,  1383,   666, 19852,   669, 10088,  1466,
          3189, 29889,   325, 29889,  3303,  3900, 29892, 29871, 29941, 29945,
         29900,   383, 29889, 29941, 29881, 29871, 29906, 29946, 29955, 29892,
         29871, 29906, 29945, 29946, 29899, 29945, 29945,   313, 29896,   303,
         25079, 29889, 29906, 29900, 29900, 29941, 29897,   313,  1054, 13081,
           292,  4251,   467,  2398, 29892,   591,   505,   451, 11527,  3692,
           278,   766,  4838,   291,   653,   740,  3682,  4988,   304,  8820,
           393,   526,  6467,  5377,  

In [179]:
out = data_collator([tokenized_datasets['train'][i] for i in range(5)])
for key in out:
    print(f'{key} shape: {out[key].shape}')

input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [28]:
from transformers import Trainer, TrainingArguments
import os
if not os.path.exists('./clmTutorial'):
    os.mkdir('./clmTutorial')
    
trainer = Trainer(
    model = model,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['valid'],
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        per_device_eval_batch_size = 32,
        gradient_accumulation_steps = 2,
        warmup_steps = 100,
        num_train_epochs = 1,
        learning_rate = 0.0002,
        logging_steps = 20,
        evaluation_strategy = "steps",
        save_strategy = "steps",
        eval_steps = 200,
        save_steps = 200,
        output_dir = './clmTutorial',
        save_total_limit = 100,
        load_best_model_at_end = True,
        ),
    data_collator = data_collator
    )


In [191]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 8442
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 815
    })
})

In [29]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=131, training_loss=1.661837279341603, metrics={'train_runtime': 524.7868, 'train_samples_per_second': 15.972, 'train_steps_per_second': 0.25, 'total_flos': 4.253395521739162e+16, 'train_loss': 1.661837279341603, 'epoch': 1.0})

In [30]:
model.save_pretrained('./clmTutorial')

/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py:1690: UserWarning: You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected behaviors. If you want to save 8-bit models, make sure to have `bitsandbytes>0.37.2` installed.
  warnings.warn(


### 2. Evaluate the model through pipelines

In [34]:
import torch
from transformers import pipeline
device = torch.device('cuda')
pipe = pipeline(
    'text-generation')

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [37]:
pipe(['hi, my name is Lora'])
pipe(['you suck at'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[{'generated_text': 'you suck at playing games. I wouldn\'t call myself a "player" if I didn\'t care for making a living as a gamer. I know I\'m not great at doing that, but when I\'m ready, I can do my job.'}]]

In [51]:
prompt = "Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.\nExcerpt: been very emphatic that, except in cases where only one inference can be drawn from the facts, negligence, proximate cause, and foreseeability are questions of fact for the jury. See St. Clair v. Denny, 245 Kan. 414, 781 P.2d 1043, 1047 (1989); Baker v. City of Garden City, 240 Kan. 554, 731 P.2d 278, 281 (1987); Gard, 400 P.2d at 1000, 1002; Rowell, 176 P.2d at 595. As might be expected in an area of the law in which so much depends on the factual scenario of the particular case, the holdings of the Kansas cases that address causation are not clearly dispositive of the issue. However, we find that those cases whose facts most closely approximate those of the instant case support the conclusion that the district court improperly granted summary judgment. See Steele, 327 P.2d at 1065 \nChoices: ['0: holding that in a  1983 action issue of probable cause is for the jury', '1: recognizing the cause of action', '2: recognizing the availability of a cause of action by manufacturer against supplier under consumer fraud act', '3: recognizing cause of action', '4: holding that demurrer improperly granted because dropping bottle of flammable liquid was reasonably probable such that it was not an intervening cause in action against bottle manufacturer']"
inputs = tokenizer(prompt, return_tensors='pt')

In [60]:
inputs

{'input_ids': tensor([[    1,  3529,  9076,   278,  1494,   429,  2265,   415,   515,   278,
          8973, 10608,   363,   278,  1206, 29889,  3575,  3414,   338,   304,
          6755,   278,  1556,  8210,  3273, 15837,   310,   278, 11706,   364,
         19478,   393, 18509,   583,   278, 16180,  1602, 12112,  8018,   304,
           278,  1206, 29889,    13,  1252,  2265,   415, 29901,  1063,  1407,
           953,   561,  2454,   393, 29892,  5174,   297,  4251,   988,   871,
           697, 27262,   508,   367, 12061,   515,   278, 17099, 29892,  3480,
          3473,   663, 29892, 23203,   403,  4556, 29892,   322,   363,   968,
         29872,  3097,   526,  5155,   310,  2114,   363,   278,   432,  2857,
         29889,  2823,   624, 29889,  6015,   381,   325, 29889,  3384,  1460,
         29892, 29871, 29906, 29946, 29945, 11720, 29889, 29871, 29946, 29896,
         29946, 29892, 29871, 29955, 29947, 29896,   349, 29889, 29906, 29881,
         29871, 29896, 29900, 29946, 2

In [42]:
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens = True, clean_up_tokenization_sapces=False)[0]

'Considering the given context, '

In [44]:
generate_ids

tensor([[    1, 10056,   292,   278,  2183,  3030, 29892, 29871,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])

In [55]:
#model_l = LlamaForCausalLM.from_pretrained('./llama')
generate_ids = model_l.generate(inputs.input_ids, max_length = 800)
tokenizer.batch_decode(generate_ids, skip_special_token = True, clean_up_tokenization_spaces = False)[0]

"<s> Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.\nExcerpt: been very emphatic that, except in cases where only one inference can be drawn from the facts, negligence, proximate cause, and foreseeability are questions of fact for the jury. See St. Clair v. Denny, 245 Kan. 414, 781 P.2d 1043, 1047 (1989); Baker v. City of Garden City, 240 Kan. 554, 731 P.2d 278, 281 (1987); Gard, 400 P.2d at 1000, 1002; Rowell, 176 P.2d at 595. As might be expected in an area of the law in which so much depends on the factual scenario of the particular case, the holdings of the Kansas cases that address causation are not clearly dispositive of the issue. However, we find that those cases whose facts most closely approximate those of the instant case support the conclusion that the district court improperly granted summary judgment. See 

In [58]:
inputs

{'input_ids': tensor([[    1,  3529,  9076,   278,  1494,   429,  2265,   415,   515,   278,
          8973, 10608,   363,   278,  1206, 29889,  3575,  3414,   338,   304,
          6755,   278,  1556,  8210,  3273, 15837,   310,   278, 11706,   364,
         19478,   393, 18509,   583,   278, 16180,  1602, 12112,  8018,   304,
           278,  1206, 29889,    13,  1252,  2265,   415, 29901,  1063,  1407,
           953,   561,  2454,   393, 29892,  5174,   297,  4251,   988,   871,
           697, 27262,   508,   367, 12061,   515,   278, 17099, 29892,  3480,
          3473,   663, 29892, 23203,   403,  4556, 29892,   322,   363,   968,
         29872,  3097,   526,  5155,   310,  2114,   363,   278,   432,  2857,
         29889,  2823,   624, 29889,  6015,   381,   325, 29889,  3384,  1460,
         29892, 29871, 29906, 29946, 29945, 11720, 29889, 29871, 29946, 29896,
         29946, 29892, 29871, 29955, 29947, 29896,   349, 29889, 29906, 29881,
         29871, 29896, 29900, 29946, 2

In [63]:
import ray
# pipeline
def make_dataset(data_dir):
    # build dataset
    path = data_dir

    # load file
    if os.path.isfile(os.path.join(path, 'prompt.pkl')):
        with open(os.path.join(path, 'prompt.pkl'), 'rb') as f:
            dataset = pickle.load(f)
    else:
        dataset_list = build_dataset(['case_hold'], path)
        dataset = dataset_list[0]['test']
        random.seed(7)
        dataset = dataset.shuffle().map(prompt_engineering).select_columns(['question', 'label'])
        
        # save file
        with open(os.path.join(path, 'prompt.pkl'), 'wb') as f:
            pickle.dump(dataset, f)

    return dataset

def prompt_engineering(data_point):

    prompt_cands = [
    "Please select the most suitable summary of the legal ruling that accompanies the relevant referenced decisions for the specific case. The following excerpt is from the court's decision.",
    "Kindly choose the concise summary of the legal ruling that accompanies the relevant referenced decisions applicable to the given case. Provided below is an excerpt from the court decision.",
    "Please decide on the most appropriate summary of the legal ruling that accompanies the relevant referenced decisions, which are relevant to the given case. Here is an excerpt from the court decision for your consideration.",
    "Here is an excerpt from the court decision for the case. Please choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Consider the following excerpt from the court decision for the case. Your task is to select the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Please refer to the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case, using the excerpt from the court decision provided below.",
    "Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Here is the excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case."
    ] # generated by Chat-GPT

    
    _slice = data_point['context'].find('(<HOLDING>)')
    excerpt = data_point['context'][:_slice]
    choices = [str(n) + f': {c}' for n,c in enumerate(data_point['endings'])]
    prompt = random.choice(prompt_cands)
    input = prompt + f'\nExcerpt: {excerpt}' + f'\nChoices: {choices}'

    return {
        'question' : input,
        'label' : data_point['label']
    }

@ray.remote(num_gpus = 1)
def tokenize(question, tokenizer):
    inputs = tokenizer(
        question,
        return_tensors = 'pt'
    )
    return inputs

@ray.remote(num_gpus = 1)
@torch.inference_mode()
def get_model_answers(model_id, inputs): # questions: {'questions': input}

    base_path = './llama'

    # load tokenizer, model
    tokenizer = LlamaTokenizer.from_pretrained(
        base_path,
        add_eos_token = True # add end-of-sentence token
    )

    adapter_names = ['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal']
    adapter_name = adapter_names[model_id]
    adapter_path = './adapter/' + adapter_name

    base_model = LlamaForCausalLM.from_pretrained(
        base_path,
        device_map = 'auto',
        low_cpu_mem_usage = True,
        torch_dtype = torch.float16
    )
    model = PeftModel.from_pretrained(
        base_model,
        adapter_path,
        torch_dtype = torch.float16,
    )
    model = model.merge_and_unload()
    
    # get answers
    output_ids = model.generate(
        torch.as_tensor(inputs.input_ids).cuda(),
        max_gen_len = 0.8,
        temperature = 0.95, # float
    )[0][len(inputs.input_ids):] # only take the answer, not the question itself

    answers = tokenizer.decode(output_ids)

    return answers

def calculate_metric(answers, labels):
    
    incorrect_correct = torch.tensor([0,0])
    labels = torch.tensor([0,0,0,0,0])

    for answer, label in enumerate(answers, labels):
        r = int(label in answer) # incorrect: 0, correct: 1 
        incorrect_correct[r] += 1
        labels[label] += 1
    
    return incorrect_correct, labels

In [ ]:
# define model
model_name = ['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][0]
# split question into num_gpus files
tokenizer = LlamaTokenizer.from_pretrained(
    base_path,
    add_eos_token = True # add end-of-sentence token
)

inputs = tokenize(datasets['question'])
labels = datasets['label']

chunk_size = len(datasets['label']) // num_gpus

for i in range(0, len(datasets['label']), chunk_size) :
    get_model_answers(model_id, inputs).remote()


results, labels = calculate_metric(answers, labels).to_list()
accuracy = results[1]/(results[0]+results[1])

print(f'The accuracy of the {model_name} is : {accuracy}')


### 3. ray module

In [3]:
import ray
from dataset import *
import pickle

In [4]:
with open('./dataset/prompt.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [5]:
dataset

Dataset({
    features: ['label', 'question'],
    num_rows: 3600
})

In [64]:
@ray.remote(num_gpus = 2)
def tokenize(question, tokenizer):
    inputs = tokenizer(
        question,
        max_length = 1024,
        padding = "max_length"
    )
    return inputs

In [65]:
ray.shutdown()
ray.init()

2023-06-11 15:01:44,427	INFO worker.py:1636 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.5.0


In [8]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [66]:
# define tokenizer, model
tokenizer = LlamaTokenizer.from_pretrained(
    './llama',
    add_eos_token = True # add end-of-sentence token
)
tokenizer.pad_token_id = 0

model = LlamaForCausalLM.from_pretrained(
    './llama',
    device_map = 'auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [67]:
inputs = ray.get(tokenize.remote(dataset['question'], tokenizer))

In [68]:
len(inputs['input_ids'][3])

1024

In [69]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [70]:
len(dataset['label'])

3600

In [ ]:
def evaluate(model_id, datasets, answer_path, num_gpus):
    # define model
    model_name = ['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]
    
    # split question into num_gpus files
    inputs = tokenize(datasets['question'])
    labels = datasets['label']

    chunk_size = len(datasets['label']) // num_gpus
    
    for i in range(0, len(datasets['label']), chunk_size) :
        get_model_answers(model_id, inputs).remote()


    results, labels = calculate_metric(answers, labels).to_list()
    accuracy = results[1]/(results[0]+results[1])

    print(f'The accuracy of the {model_name} is : {accuracy}')

In [42]:
input_ids = ray.put(inputs['input_ids'])

In [43]:
input_ids

ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000008000000)

In [ ]:
import datetime
past_datetime = datetime.datetime.now()
current_datetime = datetime.datetime.now()
time = current_datetime - past_datetime

In [71]:
ray.shutdown()
ray.init()

2023-06-11 15:02:40,351	INFO worker.py:1636 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.5.0


In [76]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [89]:
# define model
# adapter_names = ['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal']
# adapter_name = adapter_names[model_id]
adapter_path = './checkpoints_unmasked'

base_path = './llama'
base_model = LlamaForCausalLM.from_pretrained(
    base_path,
    device_map = 'auto',
    low_cpu_mem_usage = True,
    torch_dtype = torch.float16
)
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    torch_dtype = torch.float16,
)
model = model.merge_and_unload()
model.cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [81]:
import torch
from peft import PeftModel
# @ray.remote(num_gpus = 2)
# @torch.inference_mode()
def get_model_answers(model_id, input_ids, tokenizer): # questions: {'questions': input}


    
    # get answers
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        max_new_tokens = 1024,
        temperature = 0.95, # float
    )[0][len(inputs.input_ids):] # only take the answer, not the question itself

    answers = tokenizer.decode(output_ids)

    return answers

In [87]:
answers = get_model_answers(0, inputs['input_ids'][:3], tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [86]:
torch.as_tensor([inputs['input_ids'][0]]).cuda()[:,-1]

tensor([0], device='cuda:0')

In [73]:
import datetime
past_datetime = datetime.datetime.now()
answers = ray.get(get_model_answers.remote(0, inputs['input_ids'], tokenizer))
current_datetime = datetime.datetime.now()
time = current_datetime - past_datetime

2023-06-11 15:02:46,257	WARNING worker.py:2019 -- Warning: The remote function __main__.get_model_answers is very large (15 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


(pid=1603374) 
(pid=1603374) ===================================BUG REPORT===================================
(pid=1603374) Welcome to bitsandbytes. For bug reports, please run
(pid=1603374) 
(pid=1603374) python -m bitsandbytes
(pid=1603374) 
(pid=1603374)  and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
(pid=1603374) ================================================================================
(pid=1603374) bin /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
(pid=1603374) CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
(pid=1603374) CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
(pid=1603374) CUDA SETUP: Highest compute capability among GPUs detected: 8.6
(pid=1603374) CUDA SETUP: Detected CUDA version 117
(pid=1603374) CUDA SETUP: Loading binary /home/laal_intern003/a

(pid=1603374) /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/laal_intern003/anaconda/envs/legal-master did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
(pid=1603374)   warn(msg)
(pid=1603374) /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
(pid=1603374)   warn(msg)
(pid=1603374) /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/run/user/10100/vscode-ipc-59cf26d7-3f13-45b8-ae2c-a7bd3be74a22.sock')}
(pid=1603374)   warn(msg)
(pid=1603374

RayTaskError(ValueError): [36mray::get_model_answers()[39m (pid=1603374, ip=192.168.0.104)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/tmp/ipykernel_1530658/2231384656.py", line 27, in get_model_answers
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/generation/utils.py", line 1267, in generate
    self._validate_model_kwargs(model_kwargs.copy())
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/generation/utils.py", line 1140, in _validate_model_kwargs
    raise ValueError(
ValueError: The following `model_kwargs` are not used by the model: ['max_gen_len'] (note: typos in the g

In [ ]:
import torch
import shortuuid
from peft import PeftModel

# define model
adapter_path = './checkpoints_unmasked'

base_path = './llama'
base_model = LlamaForCausalLM.from_pretrained(
    base_path,
    device_map = 'auto',
    low_cpu_mem_usage = True,
    torch_dtype = torch.float16
)
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    torch_dtype = torch.float16,
)
model = model.merge_and_unload()
    
@ray.remote(num_gpus = 2)
@torch.inference_mode()
def get_model_answers_2(model, input_ids, tokenizer): # questions: {'questions': input} 
    # get answers
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        max_gen_len = 0.8,
        temperature = 0.95, # float
    )[0][len(inputs.input_ids):] # only take the answer, not the question itself

    answers = tokenizer.decode(output_ids)

    return answers

In [90]:
tokenizer(['hello']).input_ids

[[1, 22172, 2]]

In [98]:
import tqdm
def run_eval(model_id, dataset, answer_path, num_gpus):
    dataset = make_dataset('./dataset')
    question = dataset.select_columns(['question', 'idx'])
    
    chunk_size = len(question) // num_gpus
    ans_handles = []
    
    for i in range(0, len(question), chunk_size):
        ans_handles.append(
        get_model_answers.remote(
            model_id, question[i : i + chunk_size]
            )
        )
    answers = []
    
    for ans_handle in ans_handles:
        answers.extend(ray.get(ans_handle))
        
    with open('./dataset/answers', 'wb') as f:
        pickle.dump(answers, f)

@ray.remote(num_gpus = 2)
@torch.inference_mode()
def get_model_answers(model_id, questions):
    tokenizer = LlamaTokenizer.from_pretrained(
        './llama',
        add_eos_token = True # add end-of-sentence token
    )
    tokenizer.pad_token_id = 0

    model = LlamaForCausalLM.from_pretrained(
        './llama',
        device_map = 'auto').cuda()
    
    answers = []
    for i, question in enumerate(tqdm(questions)):
        prompt = question['question']
        input_ids = tokenizer([prompt]).input_ids # [[]]
        output_ids = model.generate(
            torch.as_tensor(input_ids).cuda(),
            temperature = 0.7,
            max_new_tokens = 1024,
        )
        output_ids = output_ids[0][len(input_ids[0]):] # only take the answer
        outputs = tokenizer.decode(output_ids, skip_special_tokens = True).strip()
        
        ans_id = shortuuid.uuid()
        answers.append(
            {
                "question_id": question['idx'],
                "answer": outputs,
                "answer_id": ans_id,
                "model_id": model_id,
            })
        
        return answers
    

In [93]:
def make_dataset(data_dir):
    # build dataset
    path = data_dir

    # load file
    if os.path.isfile(os.path.join(path, 'prompt.pkl')):
        with open(os.path.join(path, 'prompt.pkl'), 'rb') as f:
            dataset = pickle.load(f)
    else:
        dataset_list = build_dataset(['case_hold'], path)
        dataset = dataset_list[0]['test']
        random.seed(7)
        dataset = dataset.shuffle().map(prompt_engineering).select_columns(['question', 'label'])
        dataset = dataset.add_column(name = 'idx', column = range(3600))
        # save file
        with open(os.path.join(path, 'prompt.pkl'), 'wb') as f:
            pickle.dump(dataset, f)

    return dataset

def prompt_engineering(data_point):

    prompt_cands = [
    "Please select the most suitable summary of the legal ruling that accompanies the relevant referenced decisions for the specific case. The following excerpt is from the court's decision.",
    "Kindly choose the concise summary of the legal ruling that accompanies the relevant referenced decisions applicable to the given case. Provided below is an excerpt from the court decision.",
    "Please decide on the most appropriate summary of the legal ruling that accompanies the relevant referenced decisions, which are relevant to the given case. Here is an excerpt from the court decision for your consideration.",
    "Here is an excerpt from the court decision for the case. Please choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Consider the following excerpt from the court decision for the case. Your task is to select the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Please refer to the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case, using the excerpt from the court decision provided below.",
    "Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Here is the excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case."
    ] # generated by Chat-GPT

    
    _slice = data_point['context'].find('(<HOLDING>)')
    excerpt = data_point['context'][:_slice]
    choices = [str(n) + f': {c}' for n,c in enumerate(data_point['endings'])]
    prompt = random.choice(prompt_cands)
    input = prompt + f'\nExcerpt: {excerpt}' + f'\nChoices: {choices}'

    return {
        'question' : input,
        'label' : data_point['label']
    }

In [99]:
import random
run_eval(0, dataset, './dataset', 2)

RayTaskError(RuntimeError): [36mray::get_model_answers()[39m (pid=1608809, ip=192.168.0.104)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/tmp/ipykernel_1530658/3220729133.py", line 32, in get_model_answers
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py", line 2704, in from_pretrained
    max_memory = get_balanced_memory(
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/accelerate/utils/modeling.py", line 483, in get_balanced_memory
    max_memory = get_max_memory(max_memory)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/accelerate/utils/modeling.py", line 391, in get_max_memory
    _ = torch.tensor([0], device=i)
RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at s

2023-06-11 16:03:00,694	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::get_model_answers() (pid=1608918, ip=192.168.0.104)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/tmp/ipykernel_1530658/3220729133.py", line 32, in get_model_answers
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py", line 2704, in from_pretrained
    max_memory = get_balanced_memory(
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/accelerate/utils/modeling.py", line 483, in get_balanced_memory
    max_memory = get_max_memory(max_memory)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/accelerate/utils/modeling.py", line 391, in get_max_memory
    _ = torch.tensor([0], device=i)
RuntimeError: CUDA err

In [3]:
# import modules
import argparse
import logging
import random
import pickle
import ray
import tqdm
import shortuuid
import pandas as pd
import numpy as np
import os

## dataset
from utils.dataset import *

## models
import torch
import torch.nn as nn
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


# pipeline
def make_dataset(data_dir):
    # build dataset
    path = data_dir

    # load file
    if os.path.isfile(os.path.join(path, 'prompt.pkl')):
        with open(os.path.join(path, 'prompt.pkl'), 'rb') as f:
            dataset = pickle.load(f)
    else:
        dataset_list = build_dataset(['case_hold'], path)
        dataset = dataset_list[0]['test']
        random.seed(7)
        dataset = dataset.shuffle().map(prompt_engineering).select_columns(['question', 'label'])
        dataset = dataset.add_column(name = 'idx', column = range(3600))
        # save file
        with open(os.path.join(path, 'prompt.pkl'), 'wb') as f:
            pickle.dump(dataset, f)

    return dataset

def prompt_engineering(data_point):

    prompt_cands = [
    "Please select the most suitable summary of the legal ruling that accompanies the relevant referenced decisions for the specific case. The following excerpt is from the court's decision.",
    "Kindly choose the concise summary of the legal ruling that accompanies the relevant referenced decisions applicable to the given case. Provided below is an excerpt from the court decision.",
    "Please decide on the most appropriate summary of the legal ruling that accompanies the relevant referenced decisions, which are relevant to the given case. Here is an excerpt from the court decision for your consideration.",
    "Here is an excerpt from the court decision for the case. Please choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Consider the following excerpt from the court decision for the case. Your task is to select the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Please refer to the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case, using the excerpt from the court decision provided below.",
    "Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Here is the excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case."
    ] # generated by Chat-GPT

    
    _slice = data_point['context'].find('(<HOLDING>)')
    excerpt = data_point['context'][:_slice]
    choices = [str(n) + f': {c}' for n,c in enumerate(data_point['endings'])]
    prompt = random.choice(prompt_cands)
    input = prompt + f'\nExcerpt: {excerpt}' + f'\nChoices: {choices}'

    return {
        'question' : input,
        'label' : data_point['label']
    }

def run_eval(model_id, data_dir, answer_path, num_gpus = 3):
    dataset = make_dataset(data_dir)
    questions = dataset.select_columns(['question', 'idx'])

    chunk_size = len(questions) // num_gpus
    ans_handlers = []

    for i in range(0, len(questions), chunk_size):
        ans_handlers.append(
            get_model_answers.remote(
                model_id, questions[i:i+chunk_size]
            )
        )
    answers = []

    for ans_handler in ans_handlers:
        answers.extend(ray.get(ans_handler))
    answers = datasets.Dataset.from_pandas(pd.DataFrame(answers)) # convert to Dataset obj

    # save the answer
    with open(os.path.join(answer_path, 'answers.pkl'), 'wb') as f:
        pickle.dump(answers, f)

@ray.remote
@torch.inference_mode()
def get_model_answers(model_id, questions):
    
    tokenizer = LlamaTokenizer.from_pretrained(
        './llama',
        add_eos_token = True
    )
    tokenizer.pad_token_id = 0 # we will pad the sequence til the max length
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    print(f'****{device_map}')

    base = LlamaForCausalLM.from_pretrained(
        './llama',
        device_map = device_map,
        load_in_8bit = True,
    )
    #adapter_path = './adapter/'+['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]
    adapter_path = './LegalAdapterTrainig/checkpoints_unmasked'
    lora_model = PeftModel.from_pretrained(
        base,
        adapter_path,
        torch_dtype=torch.float16,
    )
    print("Applying the LoRA")
    model = lora_model.merge_and_unload().cuda()


    answers = []

    for i, question in enumerate(tqdm(questions)):
        prompt = question['question']
        input_ids = tokenizer([prompt]).input_ids # [[]]
        output_ids = model.generate(
            torch.as_tensor(input_ids).cuda(),
            temperature = 0.7, # manipulates how strict the model will follow the prompt instruction
            max_new_tokens = 1024,
        )
        output_ids = output_ids[0][len(input_ids[0]) : ] # only take the answer, not the question prompt from the outputs
        outputs = tokenizer.decode(output_ids, skip_special_tokens = True).strip()
        ans_id = shortuuid.uuid()
        answers.append(
            {
                "idx" : question['idx'],
                "answer" : outputs,
                "answer_id" : ans_id,
                "model_id" : model_id

            }
        )

        return answers


def evaluate(model_id, data_dir, answer_path, num_gpus):
    # define model
    model_name = ['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]

    # get answers
    run_eval(model_id, data_dir, answer_path, num_gpus)

    with open(os.path.join(answer_path, './answers.pkl', 'rb')) as f:
        answers = pickle.load(f).select_columns(['answer', 'idx'])
    labels = dataset.select_columns(['label', 'idx'])

    dataset = datasets.concatenate_datasets([answers, labels])

    # calculate metric
    calmet_handlers = []
    chunk_size = len(dataset) // num_gpus

    ray.shutdown()
    ray.init(num_gpus = num_gpus)

    for i in range(0, len(dataset), chunk_size):
        calmet_handlers.append(
            calculate_metric.remote(dataset[i:i+chunk_size])
            )
    
    results = [] 
    
    for calmet_handler in calmet_handlers:
        results.append(ray.get(calmet_handler)) # e.g. [[0,0,0,0,0,0,0],[0,0,0,0,0,0,0], ...]
    
    metrics = results.sum(axis = 0)
    accuracy = metrics[1] / (results[0] + results[1])
    
    print(f'The accuracy of the {model_name} is : {accuracy}')

@ray.remote
def calculate_metric(dataset):

    for i, data in enumerate(dataset):

        idx = dataset['idx']
        answer = dataset['answer']
        label = dataset['label']

        incorrect_correct_labels = np.array([0,0,0,0,0,0,0]) # one-hot-encoding: [incorrect, correct, 0,1,2,3,4 (ground truth)]

        r = int(label in answer) # incorrect: 0, correct: 1
        incorrect_correct_labels[r] += 1
        incorrect_correct_labels[label+2] += 1
    
    return incorrect_correct_labels


#def chatgpt_get_answer(gpt_key):


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir',
                        type = str,
                        default = './data/prompt',
                        help = 'Where LexGlue dataset is stored')
    parser.add_argument('--answer_dir',
                        type = str,
                        default = './data/answers',
                        help = 'Where answers from the model is stored')
    parser.add_argument('--gpu_num',
                        type = int,
                        default = 4,
                        help = 'The number of gpus to use for evaluation')
    parser.add_argument('--model_id',
                        type = int,
                        default = 0, 
                        help = '0: llama_legal, 1: llama_chat, 2: llama_legal_chat, 3: llama_chat_legal')

    args = parser.parse_args(args = [])

    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.DEBUG)

    evaluate(args.model_id, args.data_dir, args.answer_dir, 2)



(calculate_metric pid=3626970) /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(calculate_metric pid=3626970)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(calculate_metric pid=3626970) 
(calculate_metric pid=3626970) ===================================BUG REPORT===================================
(calculate_metric pid=3626970) Welcome to bitsandbytes. For bug reports, please run
(calculate_metric pid=3626970) 
(calculate_metric pid=3626970) python -m bitsandbytes
(calculate_metric pid=3626970) 
(calculate_metric pid=3626970)  and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
(calculate_metric pid=3626970) ================================================================================
(calculate_metric pid=3626970) bin /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
(calculate_metric pid=3626970) /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
(calculate_metric pid=3626970) CUDA SETUP: Loading binary /home/la

(get_model_answers pid=3626970) Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


(get_model_answers pid=3626970) ****{'': 0}


Loading checkpoint shards:   0%|          | 0/2 [00:04<?, ?it/s]


(calculate_metric pid=3626870)  [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(calculate_metric pid=3626870) ****{'': 0}
(calculate_metric pid=3626870) ****{'': 0}
(calculate_metric pid=3626870) ****{'': 0}
(calculate_metric pid=3626870) ****{'': 0}
(calculate_metric pid=3626870) ****{'': 0}
(calculate_metric pid=3626870) ****{'': 0}
(calculate_metric pid=3626870) ****{'': 0}
(calculate_metric pid=3626870) ****{'': 0}


Loading checkpoint shards:   0%|          | 0/2 [00:04<?, ?it/s] [repeated 2x across cluster]


RayTaskError(RuntimeError): [36mray::get_model_answers()[39m (pid=3626870, ip=192.168.0.107)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/tmp/ipykernel_3622728/93050764.py", line 110, in get_model_answers
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py", line 2777, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py", line 3118, in _load_pretrained_model
    new_error_msgs, offload_index, state_dict_index = _load_state_dict_into_meta_model(
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py", line 710, in _load_state_dict_into_meta_model
    set_module_8bit_tensor_to_device(
  File "/home/laal_intern00

2023-06-21 15:06:23,622	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::get_model_answers() (pid=3626970, ip=192.168.0.107)
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/tmp/ipykernel_3622728/93050764.py", line 110, in get_model_answers
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py", line 2777, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py", line 3118, in _load_pretrained_model
    new_error_msgs, offload_index, state_dict_index = _load_state_dict_into_meta_model(
  File "/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/transformers/modeling_utils.py", line 710, in _load_state_dict_into_me

In [15]:
ray.shutdown()

In [11]:
import torch
torch.cuda.is_available()
torch.cuda.device_count()

2

In [ ]:
!pip install bitsandbytes

     |███████████▏                    | 33.9 MB 3.6 MB/s eta 0:00:18

In [10]:
# import modules: utils
from calendar import EPOCH
import os
import sys
import pickle
import random
import json
import bitsandbytes as bn
import argparse

# import modules: learning
import torch
import torch.nn as nn
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)

# import modules: dataset
from datasets import load_dataset
from utils.dataset import *

MODEL = 0

def main(args):
    # parameters
    MICRO_BATCH_SIZE = args.micro_batch_size
    BATCH_SIZE = 64
    GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
    model_size = args.model_size # The size of Llama (i.e.,7B, 13B, 30B)
    EPOCHS = 1
    LEARNING_RATE = args.learning_rate
    CUTOFF_LEN = 512
    LORA_R = 8
    LORA_ALPHA = 16
    LORA_DROPOUT = 0.05
    # VAL_SET_SIZE = 2000 -- we don't need this cuz LexGLUE is already splited into train/val/train
    TARGET_MODULES = [
        "q_proj", # query projection
        "k_proj", # key projection
        "v_proj", # value projection
        "down_proj", # down projection
        "gate_proj", # gate projection
        "up_proj"   # up projection
    ]

    # load data
    task_list = args.task_list[0]
    path = args.data_dir
    dataset_list = build_dataset(task_list, path)
    dataset = dataset_list[0]
    dataset = rebuild_dataset(dataset)
    print("Datasets are loaded") # dict, {task_name:data}

    # load model
    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE",1))
    ddp = world_size != 1 # if there are multiple GPUs, we will do distributed learning
    
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)} # process id for the corresponding gpu
        GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size
    
    model = LlamaForCausalLM.from_pretrained(
        'llama',
        load_in_8bit = True, # this saves memory
        device_map = device_map)

    tokenizer = LlamaTokenizer.from_pretrained(
        'llama',
        add_eos_token = True # add end-of-sentence token
    )
    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r = LORA_R, # determines the compression rate
        lora_alpha = LORA_ALPHA,
        target_modules = TARGET_MODULES,
        lora_dropout = LORA_DROPOUT,
        bias = "none",
        task_type = "CAUSAL_LM"
        )

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    config.save_pretrained(args.output_dir)

    model = get_peft_model(model, config)
    tokenizer.pad_token_id = 0

    # count the total nums of params in the model
    total_params, params = 0, 0 
    for n, p in model.model.named_parameters():
        if any([x in n for x in ["lora"]]):
            total_params += p.numel()
        params += p.numel()
    print(
        f'Total number of parameters:{total_params // 1000 / 1000}M, rate {round(total_params / params * 100, 2)}%'
    )
    
    return model

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_size',
                        type = str,
                        default = '7B',
                        help = 'Foundation model size (i.e., 7B, 13B, 30B)')
    parser.add_argument('--micro_batch_size',
                        type = int,
                        default = 16,
                        help = 'Batch size') # The bigger, the faster
    parser.add_argument('--learning_rate',
                        type = float,
                        default = 0.0002,
                        help = 'Learning rate')
    parser.add_argument('--task_list',
                        default = ['case_hold'],
                        help = 'The list of datasets you want to download')
    parser.add_argument('--data_dir',
                        type = str,
                        default = 'data',
                        help = 'Where LexGlue dataset is stored')
    parser.add_argument('--output_dir',
                        type = str,
                        default = './checkpoints_16',
                        help = 'Where the adapter is stored')
    
    args = parser.parse_args(args = [])
    args.task_list = [['case_hold']]
    model = main(args)


Datasets are succesfully loaded


  0%|          | 0/3 [00:00<?, ?it/s]Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a/cache-9b631132480454cd.arrow
Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a/cache-819a6e663756257e.arrow
Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a/cache-c54604b7b969a144.arrow
100%|██████████| 3/3 [00:00<00:00, 236.29it/s]
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Datasets are loaded


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Total number of parameters:17.891M, rate 0.26%


In [26]:
[t.data for t in model.base_model.model.model.layers[26].self_attn.q_proj.lora_A.parameters()] # initialized loraB

[tensor([[-0.0058, -0.0076, -0.0059,  ..., -0.0132,  0.0022,  0.0089],
         [-0.0006, -0.0140,  0.0006,  ..., -0.0051,  0.0045, -0.0001],
         [-0.0077, -0.0127,  0.0097,  ..., -0.0092,  0.0074, -0.0142],
         ...,
         [-0.0060,  0.0086, -0.0096,  ..., -0.0043, -0.0104, -0.0108],
         [ 0.0061,  0.0106,  0.0141,  ..., -0.0079, -0.0135,  0.0105],
         [ 0.0038,  0.0043, -0.0086,  ...,  0.0111,  0.0105,  0.0062]],
        device='cuda:1')]

In [27]:
('base_model.model.model.layers.26.self_attn.q_proj.lora_A.default.weight', 
 tensor([[-0.0144,  0.0114, -0.0118,  ...,  0.0023,  0.0082,  0.0005],
        [-0.0023, -0.0016, -0.0012,  ...,  0.0063, -0.0027,  0.0002],
        [-0.0069, -0.0153, -0.0133,  ..., -0.0156, -0.0039, -0.0030],
        ...,
        [-0.0130, -0.0028,  0.0066,  ..., -0.0103, -0.0144, -0.0106],
        [ 0.0103,  0.0028,  0.0138,  ...,  0.0114, -0.0110, -0.0117],
        [-0.0089,  0.0137,  0.0061,  ...,  0.0134, -0.0156, -0.0115]],
       dtype=torch.float16))

NameError: name 'tensor' is not defined

In [32]:
import os
import sys
import torch
import pickle
import random
import json
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)


# Parameters
MICRO_BATCH_SIZE = 32
BATCH_SIZE = 64
size = "6B"
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 1
LEARNING_RATE = 0.0002
CUTOFF_LEN = 512
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
VAL_SET_SIZE = 2000
TARGET_MODULES = [
    "q_proj",
    "k_proj",
    "v_proj",
    "down_proj",
    "gate_proj",
    "up_proj",
]
DATA_PATH = "data/data_tmp.json"
OUTPUT_DIR = "checkpoints/{}".format(size)

if not os.path.exists("data"):
    os.makedirs("data")
# Load data
data = []
random.shuffle(data)
json.dump(data, open(DATA_PATH, "w"))
data = load_dataset("json", data_files=DATA_PATH)

# Load Model
device_map = "auto"
world_size = int(os.environ.get("WORLD_SIZE", 1))
ddp = world_size != 1
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size

model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-{}-hf".format(size),
    load_in_8bit=True,
    device_map=device_map,
)
total_params, params = 0, 0

tokenizer = LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-{}-hf".format(size), add_eos_token=True
)

model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
config.save_pretrained(OUTPUT_DIR)

model = get_peft_model(model, config)
tokenizer.pad_token_id = 0

for n, p in model.model.named_parameters():
    if any([x in n for x in ["lora"]]):
        total_params += p.numel()
    params += p.numel()

print(
    "Total number of parameters: {}M, rate: {}%".format(
        total_params // 1000 / 1000, round(total_params / params * 100, 2)
    )
)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/laal_intern003/.cache/huggingface/datasets/json/default-115c9468d204d314/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


OSError: decapoda-research/llama-6B-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [34]:
[t.data for t in model.base_model.model.model.layers[26].self_attn.q_proj.lora_B.parameters()] # initialized loraB

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:1')]

In [1]:
# import modules
import argparse
import logging
import random
import pickle
import ray
import tqdm
import shortuuid
import pandas as pd
import numpy as np
import os
import bitsandbytes as bn
import tqdm

## dataset
from utils.dataset import *
from utils.model import *

## models
import torch
import torch.nn as nn
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


# pipeline
def make_dataset(data_dir):
    # build dataset
    path = data_dir

    # load file
    if os.path.isfile(os.path.join(path, 'prompt.pkl')):
        with open(os.path.join(path, 'prompt.pkl'), 'rb') as f:
            dataset = pickle.load(f)
    else:
        dataset_list = build_dataset(['case_hold'], path)
        dataset = dataset_list[0]['test']
        random.seed(7)
        dataset = dataset.shuffle().map(prompt_engineering).select_columns(['question', 'label'])
        dataset = dataset.add_column(name = 'idx', column = range(3600))
        # save file
        with open(os.path.join(path, 'prompt.pkl'), 'wb') as f:
            pickle.dump(dataset, f)

    return dataset

def prompt_engineering(data_point):

    prompt_cands = [
    "Please select the most suitable summary of the legal ruling that accompanies the relevant referenced decisions for the specific case. The following excerpt is from the court's decision.",
    "Kindly choose the concise summary of the legal ruling that accompanies the relevant referenced decisions applicable to the given case. Provided below is an excerpt from the court decision.",
    "Please decide on the most appropriate summary of the legal ruling that accompanies the relevant referenced decisions, which are relevant to the given case. Here is an excerpt from the court decision for your consideration.",
    "Here is an excerpt from the court decision for the case. Please choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Consider the following excerpt from the court decision for the case. Your task is to select the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Please refer to the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case, using the excerpt from the court decision provided below.",
    "Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Here is the excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case."
    ] # generated by Chat-GPT

    
    _slice = data_point['context'].find('(<HOLDING>)')
    excerpt = data_point['context'][:_slice]
    choices = [str(n) + f': {c}' for n,c in enumerate(data_point['endings'])]
    prompt = random.choice(prompt_cands)
    input = prompt + f'\nExcerpt: {excerpt}' + f'\nChoices: {choices}'

    return {
        'question' : input,
        'label' : data_point['label']
    }

def run_eval(model_id, dataset, answer_path, num_gpus = 3):
    questions = dataset.select_columns(['question', 'idx'])

    # chunk_size = len(questions) // num_gpus
    # ans_handlers = []

    base_model_path = './llama'
    #adapter_path = './adapter/'+['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]
    adapter_model_path = '/home/laal_intern003/LegalMaster/LegalAdapterTraining/checkpoints_unmasked'

    tokenizer, _model, _device = load_tokenizer_and_model(base_model_path, adapter_model_path, load_8bit=True)
    device_map = "auto"
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    ddp = world_size != 1
    if ddp:
        device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
        GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size
 

    model = LlamaForCausalLM.from_pretrained('llama', device_map = device_map, torch_dtype = torch.float16)

    print(f'Device: {_device}')
    answers = get_model_answers(tokenizer, model, questions, device_map)

    # with torch.no_grad():
    #     for question in questions:
    #         answers.append(get_model_answers(tokenizer, model, question))

    answers = datasets.Dataset.from_pandas(pd.DataFrame(answers)) # convert to Dataset obj

    if not os.path.exists(answer_path):
        os.makedirs(answer_path)
    # save the answer
    with open(os.path.join(answer_path, f'answers_{model_id}.pkl'), 'wb') as f:
        pickle.dump(answers, f)
    torch.cuda.empty_cache()


def get_model_answers(tokenizer, model, questions, device_map):

    answers = []
#     print(questions)

    with torch.no_grad():

        for _idx, question in enumerate(tqdm(questions)):
            prompt = question['question']
            input_ids = tokenizer([prompt], return_tensors = 'pt')['input_ids'][:, -1024:].cuda()
            torch.cuda.empty_cache()

#             print(f'question: {question}')

            outputs = simple_decode(
                input_ids,
                model,
                tokenizer,
                max_new_tokens = 512,
            )

            ans_id = shortuuid.uuid()
            answers.append(
                {
                    "idx" : _idx,
                    "answer" : outputs,
                    "answer_id" : ans_id,
                    # "model_id" : model_id

                }
                )
            
    return answers
            

def evaluate(model_id, data_dir, answer_path, num_gpus):
    # define model

    model_name = ['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]

    # get answers
    dataset = make_dataset(data_dir)
    run_eval(model_id, dataset, answer_path, num_gpus)

    with open(os.path.join(answer_path, f'./answers_{model_id}.pkl'), 'rb') as f:
        answers = pickle.load(f).select_columns(['answer', 'idx'])

    labels = dataset.select_columns(['label', 'idx'])

    dataset = datasets.concatenate_datasets([answers, labels])

@ray.remote
def calculate_metric(dataset):

    for i, data in enumerate(dataset):

        idx = dataset['idx']
        answer = dataset['answer']
        label = dataset['label']

        incorrect_correct_labels = np.array([0,0,0,0,0,0,0]) # one-hot-encoding: [incorrect, correct, 0,1,2,3,4 (ground truth)]

        r = int(label in answer) # incorrect: 0, correct: 1
        incorrect_correct_labels[r] += 1
        incorrect_correct_labels[label+2] += 1
    
    return incorrect_correct_labels


#def chatgpt_get_answer(gpt_key):


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir',
                        type = str,
                        default = './data/prompt',
                        help = 'Where LexGlue dataset is stored')
    parser.add_argument('--answer_dir',
                        type = str,
                        default = './data/answers',
                        help = 'Where answers from the model is stored')
    parser.add_argument('--gpu_num',
                        type = int,
                        default = 2,
                        help = 'The number of gpus to use for evaluation')
    parser.add_argument('--model_id',
                        type = int,
                        default = 0, 
                        help = '0: llama_legal, 1: llama_chat, 2: llama_legal_chat, 3: llama_chat_legal')

    args = parser.parse_args(args = [])

    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.DEBUG)

    evaluate(args.model_id, args.data_dir, args.answer_dir, args.gpu_num)




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/laal_intern003/anaconda/envs/legal-master did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/run/user/10100/vscode-ipc-b71cb56e-ff7d-41c0-9383-cb7ffbe2f918.sock')}
  warn(msg)
/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cuda_se

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device: cuda
Dataset({
    features: ['question', 'idx'],
    num_rows: 3600
})


  0%|          | 0/3600 [00:00<?, ?it/s]

question: {'question': "Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.\nExcerpt: W. Reginald Rose, Jr., and Laura J. Rose appeal pro se from the district court’s order dismissing their civil rights complaint for failing to comply with Rule 8 of the Federal Rules of Civil Procedure. We dismiss the appeal for lack of jurisdiction. The district court dismissed the Roses’ complaint without prejudice and granted leave to amend. Rather than filing an amended complaint or obtaining a final order of dismissal from the district court, the Roses filed a notice of appeal. We therefore lack jurisdiction. See WMX Techs., Inc. v. Miller, 104 F.3d 1133, 1136-37 (9th Cir.1997) (en banc) \nChoices: ['0: holding that denial of a postconviction motion without prejudice and with leave to amend is not a final appealable order', '1: holding that dismissal wit

  0%|          | 0/3600 [00:32<?, ?it/s]

True
['\nThe Roses’ complaint alleges that the district court violated their due process rights by denying them a fair trial and by failing to provide them with a speedy trial. The district court dismissed the complaint without prejudice and granted the Roses leave to amend. The Roses did not file an amended complaint or obtain a final order of dismissal from the district court. Instead, they filed a notice of appeal. We therefore lack jurisdiction. See WMX Techs., Inc. v. Miller, 104 F.3d 1133, 1136-37 (9th Cir.1997) (en banc).\nThe Roses’ complaint alleges that the district court violated their due process rights by denying them a fair trial and by failing to provide them with a speedy trial. The district court dismissed the complaint without prejudice and granted the Roses leave to amend. The Roses did not file an amended complaint or obtain a final order of dismissal from the district court. Instead, they filed a notice of appeal. We therefore lack jurisdiction. See WMX Techs., Inc